In [2]:
pip install openjij

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [4]:
import itertools
import openjij as oj
from collections import defaultdict

# -------------------------
# 問題設定
# -------------------------
shops = [1, 2, 3]
car_types = ["small", "large"]
size = {"small": 1, "large": 2}

T = [0, 1, 2]       # 積載順（奥→手前）
S_max = 3           # 最大積載容量

# ペナルティ係数
A = 10.0  # 1スロット1台
B = 10.0  # サイズ付き容量
C = 5.0   # 需要
D = 5.0   # 荷崩れ

# 需要
demand = {
    (1, "small"): 1,
    (2, "large"): 1,
    (3, "small"): 1
}

# -------------------------
# QUBO定義
# -------------------------
Q = defaultdict(float)

def add_linear(var, coef):
    Q[(var, var)] += coef

def add_quadratic(var1, var2, coef):
    if var1 <= var2:
        Q[(var1, var2)] += coef
    else:
        Q[(var2, var1)] += coef

# 変数名
def y(i, k, t):
    return f"y_{i}_{k}_{t}"

def x(i, t):
    return f"x_{i}_{t}"

# -------------------------
# 制約①：各スロットに1台
# (sum y - 1)^2
# -------------------------
for t in T:
    vars_t = [y(i, k, t) for i, k in itertools.product(shops, car_types)]
    for v in vars_t:
        add_linear(v, A)
    for v1, v2 in itertools.combinations(vars_t, 2):
        add_quadratic(v1, v2, 2 * A)
    add_linear(vars_t[0], -2 * A)  # 定数項の代替（簡略）

# -------------------------
# 制約②：サイズ付き容量制約
# (sum size*y - Smax)^2
# -------------------------
for t in T:
    vars_t = [(y(i, k, t), size[k]) for i, k in itertools.product(shops, car_types)]
    for v, s in vars_t:
        add_linear(v, B * s * s)
        add_linear(v, -2 * B * S_max * s)
    for (v1, s1), (v2, s2) in itertools.combinations(vars_t, 2):
        add_quadratic(v1, v2, 2 * B * s1 * s2)

# -------------------------
# 制約③：需要制約
# (sum y - d)^2
# -------------------------
for (i, k), d in demand.items():
    vars_d = [y(i, k, t) for t in T]
    for v in vars_d:
        add_linear(v, C)
        add_linear(v, -2 * C * d)
    for v1, v2 in itertools.combinations(vars_d, 2):
        add_quadratic(v1, v2, 2 * C)

# -------------------------
# 制約④：積載と配送先の整合性
# y <= x  → (y - x)^2
# -------------------------
for i, k, t in itertools.product(shops, car_types, T):
    add_linear(y(i, k, t), C)
    add_linear(x(i, t), C)
    add_quadratic(y(i, k, t), x(i, t), -2 * C)

# -------------------------
# 制約⑤：荷崩れ（積み下ろし順）
# y(i,t1) * x(i,t2)  (t1 < t2)
# -------------------------
for i in shops:
    for t1, t2 in itertools.combinations(T, 2):
        for k in car_types:
            add_quadratic(y(i, k, t1), x(i, t2), D)

# -------------------------
# サンプリング
# -------------------------
sampler = oj.SASampler()
response = sampler.sample_qubo(Q, num_reads=100)

# -------------------------
# 結果表示
# -------------------------
best = response.first.sample
print("最良解:")
for k, v in best.items():
    if v == 1:
        print(k)


最良解:
x_1_1
x_3_2
y_1_large_1
y_1_small_1
y_2_large_0
y_3_large_2
